<a href="https://colab.research.google.com/github/WongPuiYee/MCU/blob/master/0604OOP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

我是一個開發人員，先幫我建立四個串列student_ids，names、scores和genders，分別存放使用者輸入學生的學號、學生的姓名、學生的成績、學生的性別，幫我撰寫新增、修改、刪除和查詢全部學生的學號、學生的姓名、學生的成績、學生的性別，最後寫一個新增、修改、刪除、查詢全部學生和離開的使用者介面。

我是一個開發人員，先幫我建立一個字典，該字典包含學生的學號、學生的姓名、學生的成績、學生的性別，用一個list來管理全班學生，幫我撰寫新增、修改、刪除和查詢全部學生的學號、學生的姓名、學生的成績、學生的性別，最後寫一個新增、修改、刪除、查詢全部學生和離開的使用者介面。

我是一個開發人員，先幫我建立一個學生類別，該物件包含學生的學號、學生的姓名、學生的成績、學生的性別，用一個list來管理全班學生，幫我撰寫新增、修改、刪除和查詢全部學生的學號、學生的姓名、學生的成績、學生的性別，最後寫一個新增、修改、刪除、查詢全部學生和離開的使用者介面。

In [10]:
# prompt: 我是一個開發人員，先幫我建立一個學生類別，該物件包含學生的學號、學生的姓名、學生的成績、學生的性別，用一個Sqlite來管理全班學生，幫我撰寫新增、修改、刪除和查詢全部學生的學號、學生的姓名、學生的成績、學生的性別，最後寫一個新增、修改、刪除、查詢全部學生和離開的使用者介面。

import sqlite3

class Student:
  def __init__(self, student_id, name, score, gender):
    self.student_id = student_id
    self.name = name
    self.score = score
    self.gender = gender

  def __repr__(self):
    return f"學號: {self.student_id}, 姓名: {self.name}, 成績: {self.score}, 性別: {self.gender}"

class StudentManager:
  def __init__(self, db_name="students.db"):
    self.db_name = db_name
    self._create_table()

  def _create_table(self):
    conn = sqlite3.connect(self.db_name)
    cursor = conn.cursor()
    cursor.execute('''
        CREATE TABLE IF NOT EXISTS students (
            student_id TEXT PRIMARY KEY,
            name TEXT,
            score REAL,
            gender TEXT
        )
    ''')
    conn.commit()
    conn.close()

  def add_student(self, student):
    conn = sqlite3.connect(self.db_name)
    cursor = conn.cursor()
    try:
      cursor.execute("INSERT INTO students VALUES (?, ?, ?, ?)",
                     (student.student_id, student.name, student.score, student.gender))
      conn.commit()
      print("學生新增成功！")
    except sqlite3.IntegrityError:
      print(f"學號 {student.student_id} 已存在。")
    conn.close()

  def update_student(self, student_id, name=None, score=None, gender=None):
    conn = sqlite3.connect(self.db_name)
    cursor = conn.cursor()

    # Check if student exists
    cursor.execute("SELECT 1 FROM students WHERE student_id = ?", (student_id,))
    if cursor.fetchone() is None:
      print(f"錯誤：找不到學號 {student_id} 的學生。")
      conn.close()
      return

    updates = []
    values = []
    if name is not None and name != "":
      updates.append("name = ?")
      values.append(name)
    if score is not None:
      updates.append("score = ?")
      values.append(score)
    if gender is not None and gender != "":
      updates.append("gender = ?")
      values.append(gender)

    if not updates:
      print("沒有提供要更新的資料。")
      conn.close()
      return

    query = f"UPDATE students SET {', '.join(updates)} WHERE student_id = ?"
    values.append(student_id)

    cursor.execute(query, tuple(values))
    conn.commit()
    print(f"學號 {student_id} 的資料更新成功！")
    conn.close()


  def delete_student(self, student_id):
    conn = sqlite3.connect(self.db_name)
    cursor = conn.cursor()
    cursor.execute("DELETE FROM students WHERE student_id = ?", (student_id,))
    conn.commit()
    if cursor.rowcount > 0:
      print(f"學號 {student_id} 的學生已刪除。")
    else:
      print(f"找不到學號 {student_id} 的學生。")
    conn.close()

  def get_all_students(self):
    conn = sqlite3.connect(self.db_name)
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM students")
    rows = cursor.fetchall()
    conn.close()
    students = []
    for row in rows:
      students.append(Student(row[0], row[1], row[2], row[3]))
    return students

  def get_student_by_id(self, student_id):
    conn = sqlite3.connect(self.db_name)
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM students WHERE student_id = ?", (student_id,))
    row = cursor.fetchone()
    conn.close()
    if row:
      return Student(row[0], row[1], row[2], row[3])
    return None


def display_menu():
  print("\n=== 學生管理系統 ===")
  print("1. 新增學生")
  print("2. 修改學生")
  print("3. 刪除學生")
  print("4. 查詢所有學生")
  print("5. 離開")
  print("==================")

def main():
  manager = StudentManager()

  while True:
    display_menu()
    choice = input("請輸入您的選擇: ")

    if choice == '1':
      student_id = input("請輸入學號: ")
      name = input("請輸入姓名: ")
      while True:
        try:
          score = float(input("請輸入成績: "))
          break
        except ValueError:
          print("成績必須是數字。")
      gender = input("請輸入性別: ")
      new_student = Student(student_id, name, score, gender)
      manager.add_student(new_student)

    elif choice == '2':
      student_id = input("請輸入要修改資料的學號: ")
      student_to_update = manager.get_student_by_id(student_id) # Get student data

      if student_to_update is None:
        print(f"錯誤：找不到學號 {student_id} 的學生。")
        continue # Go back to the main menu

      print("\n--- 目前學生資料 ---")
      print(student_to_update)
      print("--------------------")


      print("請輸入要修改的資料 (留空表示不修改):")
      name = input(f"新姓名 (目前: {student_to_update.name}): ")
      score_str = input(f"新成績 (目前: {student_to_update.score}): ")
      score = float(score_str) if score_str else None
      gender = input(f"新性別 (目前: {student_to_update.gender}): ")

      manager.update_student(student_id,
                             name=name if name else None,
                             score=score,
                             gender=gender if gender else None)

    elif choice == '3':
      student_id = input("請輸入要刪除的學號: ")
      manager.delete_student(student_id)

    elif choice == '4':
      students = manager.get_all_students()
      if students:
        print("\n=== 所有學生資料 ===")
        for student in students:
          print(student)
        print("===================")
      else:
        print("目前沒有學生資料。")

    elif choice == '5':
      print("謝謝使用學生管理系統，再見！")
      break

    else:
      print("無效的選擇，請重新輸入。")

if __name__ == "__main__":
  main()


=== 學生管理系統 ===
1. 新增學生
2. 修改學生
3. 刪除學生
4. 查詢所有學生
5. 離開
請輸入您的選擇: 4

=== 所有學生資料 ===
學號: 12131484, 姓名: 黃佩怡, 成績: 88.0, 性別: 女

=== 學生管理系統 ===
1. 新增學生
2. 修改學生
3. 刪除學生
4. 查詢所有學生
5. 離開
請輸入您的選擇: 2
請輸入要修改資料的學號: 1
錯誤：找不到學號 1 的學生。

=== 學生管理系統 ===
1. 新增學生
2. 修改學生
3. 刪除學生
4. 查詢所有學生
5. 離開
請輸入您的選擇: 2
請輸入要修改資料的學號: 12131484

--- 目前學生資料 ---
學號: 12131484, 姓名: 黃佩怡, 成績: 88.0, 性別: 女
--------------------
請輸入要修改的資料 (留空表示不修改):
新姓名 (目前: 黃佩怡): py
新成績 (目前: 88.0): 90
新性別 (目前: 女): 女
學號 12131484 的資料更新成功！

=== 學生管理系統 ===
1. 新增學生
2. 修改學生
3. 刪除學生
4. 查詢所有學生
5. 離開
請輸入您的選擇: 4

=== 所有學生資料 ===
學號: 12131484, 姓名: py, 成績: 90.0, 性別: 女

=== 學生管理系統 ===
1. 新增學生
2. 修改學生
3. 刪除學生
4. 查詢所有學生
5. 離開
請輸入您的選擇: 5
謝謝使用學生管理系統，再見！


--- 學生管理系統 ---
1. 新增學生
2. 修改學生
3. 刪除學生
4. 查詢所有學生
5. 離開
請選擇操作 (1-5): 4
目前沒有學生資料。

--- 學生管理系統 ---
1. 新增學生
2. 修改學生
3. 刪除學生
4. 查詢所有學生
5. 離開
請選擇操作 (1-5): 1
請輸入學號: 9701824
請輸入姓名: lendy
請輸入成績: 90
請輸入性別: 男
學生已成功新增。

--- 學生管理系統 ---
1. 新增學生
2. 修改學生
3. 刪除學生
4. 查詢所有學生
5. 離開
請選擇操作 (1-5): 2
請輸入要修改的學號: 001
錯誤：找不到指定的學號。

--- 學生管理系統 ---
1. 新增學生
2. 修改學生
3. 刪除學生
4. 查詢所有學生
5. 離開
請選擇操作 (1-5): 2
請輸入要修改的學號: 9701824

--- 修改學生資料 ---
請輸入新的姓名 (目前: lendy, 留空表示不修改): lin
請輸入新的成績 (目前: 90, 留空表示不修改): 88
請輸入新的性別 (目前: 男, 留空表示不修改):
學生資料已成功更新。

--- 學生管理系統 ---
1. 新增學生
2. 修改學生
3. 刪除學生
4. 查詢所有學生
5. 離開
請選擇操作 (1-5): 4

--- 所有學生資料 ---
學號: 9701824, 姓名: lin, 成績: 88, 性別: 男

--- 學生管理系統 ---
1. 新增學生
2. 修改學生
3. 刪除學生
4. 查詢所有學生
5. 離開
請選擇操作 (1-5): 3
請輸入要刪除的學號: 9701824
學生資料已成功刪除。

--- 學生管理系統 ---
1. 新增學生
2. 修改學生
3. 刪除學生
4. 查詢所有學生
5. 離開
請選擇操作 (1-5): 4
目前沒有學生資料。

--- 學生管理系統 ---
1. 新增學生
2. 修改學生
3. 刪除學生
4. 查詢所有學生
5. 離開
請選擇操作 (1-5): 5
程式結束。